In [1]:
from dotenv import load_dotenv
import os
from elasticsearch import Elasticsearch
import retrieve_data

# take environment variables from .env.
load_dotenv()

PROJECT_INDEX = os.getenv("PROJECT_INDEX")

In [42]:
projects = retrieve_data.run_project()
profiles = retrieve_data.run_profile()

C:\Users\aless\Desktop\testEmbeddingModels\retrieve_data.py:53: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  search_results = dev_client.search(index=PROJECT_INDEX, query=query, scroll="30s", size=10000)['hits']['hits']
C:\Users\aless\Desktop\testEmbeddingModels\retrieve_data.py:33: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  search_results = dev_client.search(index=PROFILE_INDEX, query=query, scroll="30s", size=10000)['hits']['hits']


In [43]:
profiles

[{'profile_id': 1,
  'sentence': 'My work takes inspiration from anthropology, data science and posthuman design.',
  'n_sentence': 1},
 {'profile_id': 1,
  'sentence': 'Through structured methodological research I reveal opportunities for creativity, synthesizing user insights and opening up design avenues.',
  'n_sentence': 2},
 {'profile_id': 2, 'sentence': 'bio bio bio.', 'n_sentence': 1},
 {'profile_id': 3, 'sentence': 'Hello from my bio!', 'n_sentence': 1},
 {'profile_id': 3, 'sentence': 'Edit #1.', 'n_sentence': 2},
 {'profile_id': 4, 'sentence': 'Soon.', 'n_sentence': 1},
 {'profile_id': 8,
  'sentence': 'Describe youself in a few lines.',
  'n_sentence': 1},
 {'profile_id': 8,
  'sentence': 'Used for matchmaking purposesDescribe youself in a few lines.',
  'n_sentence': 2},
 {'profile_id': 8,
  'sentence': 'Used for matchmaking purposesDescribe youself in a few lines.',
  'n_sentence': 3},
 {'profile_id': 8,
  'sentence': 'Used for matchmaking purposesDescribe youself in a few

In [78]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch
import statistics
from utilis import rebuild_bio_with_list_sentences


def semantic_text_similarity(data, sentences, model='all-MiniLM-L6-v2', type_operation="mean"):
    embedder = SentenceTransformer(model)
    final_result = dict()
    all_scores = dict()
    data = rebuild_bio_with_list_sentences(data)

    # Split QUERY and CORPUS
    corpus_info_sentences = data

    embeddings1 = embedder.encode(sentences, convert_to_tensor=True)

    # CORPUS
    for key, bio in corpus_info_sentences.items():

        sentences2 = bio
        embeddings2 = embedder.encode(sentences2, convert_to_tensor=True)
        # Compute cosine-similarities
        cosine_scores = util.cos_sim(embeddings1, embeddings2)
        
        list_of_tensors = list()
        
        for query in cosine_scores:
            list_of_tensors.append(max(query))
            
        stak_tensor = torch.stack(list_of_tensors)
        print(key)
        print(stak_tensor)

        if type_operation == 'mean':
            final_result[key] = torch.mean(stak_tensor).item()
        if type_operation == 'median':
            final_result[key] = np.median(stak_tensor)

    return final_result

In [35]:
profile_id = 2

In [53]:
from utilis import get_bio_by_profile_id, rebuild_bio, rebuild_bio_with_list_sentences
bio = rebuild_bio_with_list_sentences(profiles)[12]

In [96]:
get_bio_by_profile_id(projects, 33)

'Dancing Philosophy. The project is based on an intensive collaboration between a dancer and a philosopher to explore the question of desire, in its multiple links to practices of recognition.\n It also associates a choreographer, specialised in Laban Kinetography and expert in Functional Analysis of the Body in Dancing Movement, a performance artist, director and graphic designer as well as a professor in performing arts and digital humanities.\n Through this collaboration, we would like to develop analytical tools and pedagogical practices that allow an innovative approach to provoke the individual’s recognition of “self” in relation to “other”.   The idea is to work together first in a dialectical exchange on the points and issues of the main theme. Then, we will take the materials and stories that have emerged and examine them in an improvisational work where the “body in movement” becomes the subject, the object and the tool of the research. The idea is to move from isolated thoug

In [54]:
print(bio)

['I am a historian working in Lisbon.', 'Currently I hold a post-doctoral fellowship in the H2020 project “COESO:Collaborative Engagements in Societal Issues” at the Centre for Research in Anthropology (University Institute of Lisbon);\nI am about to start an individual research project (2022-2028)', '“Out of Work: a critical history of unemployment in Portugal, 1890s-1970s”, at the Institute of Contemporary History (NOVA University of\nLisbon, Portugal).', 'My research focuses on the fields of Social History, Southern European History after my PhD (2020).', 'I co-funded', 'and I am editor of\xa0Práticas da História.', 'Journal on Theory, Historiography and Uses of the Past.']


In [94]:
result = semantic_text_similarity(data, bio, model='labse')

1
tensor([0.2599, 0.3409, 0.2771, 0.1930, 0.0399, 0.2272, 0.2233])
2
tensor([0.2805, 0.2939, 0.2946, 0.2201, 0.0140, 0.3178, 0.2812])
3
tensor([ 0.2778,  0.2653,  0.2978,  0.2127, -0.0025,  0.3123,  0.2752])
4
tensor([ 0.2100,  0.2181,  0.2339,  0.2691, -0.0395,  0.1627,  0.2504])
9
tensor([0.2804, 0.2352, 0.2433, 0.1868, 0.0921, 0.2039, 0.2696])
10
tensor([0.1342, 0.1679, 0.2118, 0.1903, 0.0477, 0.1343, 0.1312])
11
tensor([0.2273, 0.3571, 0.4264, 0.4394, 0.0864, 0.2542, 0.3096])
12
tensor([0.2617, 0.2598, 0.3192, 0.2640, 0.1170, 0.1879, 0.2137])
13
tensor([0.5331, 0.2536, 0.3837, 0.1384, 0.0585, 0.1105, 0.2060])
14
tensor([0.2804, 0.3993, 0.3069, 0.3291, 0.1415, 0.2964, 0.1807])
15
tensor([0.2076, 0.2265, 0.2270, 0.2163, 0.1449, 0.2776, 0.1871])
18
tensor([0.2189, 0.2853, 0.2646, 0.1873, 0.0609, 0.2052, 0.2197])
19
tensor([ 0.1948,  0.2796,  0.2803,  0.2096, -0.0049,  0.2337,  0.2242])
20
tensor([0.2535, 0.4685, 0.4629, 0.3875, 0.1097, 0.3158, 0.2503])
21
tensor([0.2318, 0.4386, 0.398

In [95]:
dict(sorted(result.items(), key=lambda item: item[1], reverse=True))

{43: 0.3527189791202545,
 33: 0.3513282239437103,
 29: 0.3440079689025879,
 42: 0.33223143219947815,
 20: 0.32116225361824036,
 24: 0.306226521730423,
 34: 0.30073586106300354,
 11: 0.3000558018684387,
 31: 0.2928733229637146,
 21: 0.2919839322566986,
 36: 0.2855660319328308,
 49: 0.28159403800964355,
 48: 0.28143763542175293,
 23: 0.28121286630630493,
 32: 0.28015393018722534,
 14: 0.27633264660835266,
 56: 0.2658243775367737,
 54: 0.25821653008461,
 57: 0.25618788599967957,
 26: 0.24675722420215607,
 25: 0.2444520741701126,
 2: 0.24316681921482086,
 13: 0.24053855240345,
 3: 0.23408500850200653,
 12: 0.23188552260398865,
 52: 0.2309182733297348,
 53: 0.22543776035308838,
 1: 0.22304005920886993,
 47: 0.22000433504581451,
 9: 0.2158965915441513,
 15: 0.2124347686767578,
 22: 0.20718105137348175,
 18: 0.20599280297756195,
 55: 0.20453934371471405,
 19: 0.2024545967578888,
 45: 0.20113159716129303,
 4: 0.18638017773628235,
 40: 0.18168891966342926,
 30: 0.1790558099746704,
 28: 0.176206